# Analyse de Data Drift avec Evidently

In [6]:
import pandas as pd
from evidently import Report

from zipfile import ZipFile
from pathlib import Path

from evidently.presets import DataDriftPreset

# Chemin vers le zip téléchargé (adapte si nécessaire)
archive_folder = Path('..')

# Chargement des datasets
app_train_df = pd.read_csv(archive_folder / 'application_train.csv')
app_test_df = pd.read_csv(archive_folder / 'application_test.csv')

# Hypothèse : app_train est la référence (données d'entraînement)
# et app_test simule les nouvelles données en production
reference_data = app_train_df.sample(n=20000, random_state=42)
current_data = app_test_df.sample(n=20000, random_state=42)

print("Données de référence chargées:", reference_data.shape)
print("Données courantes chargées:", current_data.shape)

(run := Report(metrics=[DataDriftPreset(), ]).run(reference_data=reference_data, current_data=current_data)).save_html(
    'report.html')

run

Données de référence chargées: (20000, 122)
Données courantes chargées: (20000, 121)
